In [1]:
# useing this variable for saving models and summaries
exp_name = "CNN9005_opp"

## Importing Libraries

In [2]:
%load_ext autoreload
%autoreload 2
import tensorflow as tf
import numpy as np
from hdf5storage import savemat
import hickle 

import sys
sys.path.append('../src')

In [3]:
import models.CNN_model as CNN_model
import training
import evaluation.nn_eval_activity as nn_eval

## Loading synthetic data

In [4]:
#loading data
data_path= '../data/opp6.hkl'
with open(data_path, 'r') as fin:
    data = hickle.load(fin)

In [5]:
train_data = data['train_data']
validation_data = data['validation_data']
test_data = data['test_data']

train_gt = data['train_gt']
validation_gt = data['validation_gt']
test_gt = data['test_gt']

In [6]:
train_data.shape, train_gt.shape

((1358, 1024, 77), (1358, 1024, 18))

In [7]:
test_data.shape, test_gt.shape

((233, 1024, 77), (233, 1024, 18))

## Model

In [11]:
# Model parameters
n_variables = 77
learning_rate = 0.001
batch_size = 100

convolution_levels = 3    #number of convolution to have in the network
pooling_stride = [4, 4, 4]
weights_shape = [[9, n_variables, 32],
                 [5, 32, 32],
                 [5, 32, 32]]
bias_shape = [32, 32, 32]
activation = tf.nn.relu
n_classes = 18

##positive weight for weighted cross entropy
s = train_gt.shape
resolution = np.prod(pooling_stride)

print s[0]*s[1], (s[0]*s[1]/resolution), (s[0]*s[1]/resolution)*resolution

pooled_gt = np.reshape(train_gt[:,:(s[0]*s[1]/resolution)*resolution,:], (s[0]*(s[1]/resolution), resolution, s[2])).max(axis=1)
pos_weight = (1.0 - pooled_gt.mean(axis = 0)) / pooled_gt.mean(axis=0)


1390592 21728 1390592


In [12]:
pos_weight.shape

(18,)

In [13]:
pos_weight

array([  0.1575919 ,  33.59872611,  33.82051282,  36.46206897,
        35.45637584,  26.22807018,  26.78516624,  39.3866171 ,
        41.27237354,  45.82758621,  50.98086124,  47.71748879,
        50.24528302,  42.63052209,  43.89256198,  33.16352201,
        10.44783983,  35.95238095])

In [14]:
# Launch the graph
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

In [15]:
summary_path = '../summary/' + exp_name 
checkpoint_path = '../model/' + exp_name + '/' + exp_name 

In [16]:
model = CNN_model.Model(n_variables = n_variables,
                        learning_rate = learning_rate,
                        n_classes = n_classes)

In [17]:
model.build_conv_net(pos_weight_values = pos_weight,
                     learning_rate = learning_rate,
                     convolution_levels = convolution_levels, 
                     pooling_stride = pooling_stride,
                     weights_shape = weights_shape,
                     bias_shape = bias_shape,
                     n_classes = n_classes,
                     activation = activation)

In [18]:
trainer = training.Trainer(model = model,
                    data_train = train_data,
                    ground_truth_train = train_gt,
                    data_validation = test_data,
                    ground_truth_validation = test_gt,
                    sess = sess,
                    summary_path = summary_path,
                    batch_size = batch_size,
                    global_step = 0,
                    eval_type = 'activity')

In [19]:
trainer.train(max_iter = 20000,
                train_eval_step = 100, 
                validation_eval_step = 100,
                display_step = 100)

Train Batch Evaluation
AUC = 0.076 vs. 0.079
F1 = 0.124 vs. 0.119
Accuracy = 0.025 vs. 0.056
Validation
AUC = 0.067 vs. 0.068
F1 = 0.096 vs. 0.093
Accuracy = 0.011 vs. 0.056
Minibatch Loss= 1.454656
Train Batch Evaluation
AUC = 0.610 vs. 0.082
F1 = 0.622 vs. 0.120
Accuracy = 0.698 vs. 0.056
Validation
AUC = 0.387 vs. 0.069
F1 = 0.451 vs. 0.093
Accuracy = 0.681 vs. 0.056
Minibatch Loss= 0.390944
Train Batch Evaluation
AUC = 0.706 vs. 0.078
F1 = 0.696 vs. 0.113
Accuracy = 0.788 vs. 0.056
Validation
AUC = 0.462 vs. 0.071
F1 = 0.533 vs. 0.099
Accuracy = 0.735 vs. 0.056
Minibatch Loss= 0.218345
Train Batch Evaluation
AUC = 0.831 vs. 0.084
F1 = 0.818 vs. 0.131
Accuracy = 0.815 vs. 0.056
Validation
AUC = 0.478 vs. 0.069
F1 = 0.548 vs. 0.095
Accuracy = 0.781 vs. 0.056
Minibatch Loss= 0.177516
Train Batch Evaluation
AUC = 0.875 vs. 0.078
F1 = 0.853 vs. 0.116
Accuracy = 0.866 vs. 0.056
Validation
AUC = 0.511 vs. 0.067
F1 = 0.564 vs. 0.091
Accuracy = 0.810 vs. 0.056
Minibatch Loss= 0.120122
Train

KeyboardInterrupt: 

In [20]:
#Save the model
saver = tf.train.Saver()
saver.save(sess, checkpoint_path, global_step=trainer.global_step)

'../model/CNN9005_opp/CNN9005_opp-8344'

In [ ]:
#load the model
saver = tf.train.Saver()
#saver.restore(sess, "../model" + "/CNN7002_Gleam" + "/CNN7002_Gleam")
saver.restore(sess, '../model/CNN7002_Gleam/CNN7002_Gleam-10000')


In [ ]:
#This is the model evaluation on test dataset
_= nn_eval.evaluate_model (model, sess, test_data, test_gt)

In [ ]:
#shape of probability_change is (batch size, T, num_classes)
#since num_classes = 1 we squeeze that axis
probability_change = np.squeeze(probability_of_pos, axis = 2)

In [ ]:
savemat('../poc/poc'+exp_name+'.mat', {'poc':probability_change})

In [ ]:
1